# Purpose

2021-09-28: Run inference on posts for v0.4.0 POSTS.

Diff from before: instead of only using `text` (post title + post body), we'll be using multiple columns to concat and get the embeddings.

---

This notebook runs the `vectorize_text_to_embeddings` function to:
- loading USE-multilingual model
- load post & comment text
- convert the text into embeddings (at post or level)


# Notebook setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime
import gc
# from functools import partial
# import os
import logging
# from pathlib import Path
# from pprint import pprint

import mlflow

# from tqdm.auto import tqdm
from tqdm import tqdm
import numpy as np
import pandas as pd

from google.cloud import storage

# TF libraries... I've been getting errors when these aren't loaded
import tensorflow_text
import tensorflow as tf

import subclu
from subclu.utils.hydra_config_loader import LoadHydraConfig
from subclu.models.vectorize_text import (
    vectorize_text_to_embeddings,
)
from subclu.models import vectorize_text_tf

from subclu.utils import set_working_directory
from subclu.utils.mlflow_logger import MlflowLogger
from subclu.utils.eda import (
    setup_logging, counts_describe, value_counts_and_pcts,
    notebook_display_config, print_lib_versions,
    style_df_numeric
)


print_lib_versions([mlflow, np, mlflow, pd, tensorflow_text, tf, subclu])

python		v 3.7.10
===
mlflow		v: 1.16.0
numpy		v: 1.18.5
mlflow		v: 1.16.0
pandas		v: 1.2.5
tensorflow_text	v: 2.3.0
tensorflow	v: 2.3.3
subclu		v: 0.4.0


In [3]:
# plotting
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.dates as mdates
plt.style.use('default')

setup_logging()
notebook_display_config()

# Initialize mlflow logging with sqlite database

In [4]:
# use new class to initialize mlflow
mlf = MlflowLogger(tracking_uri='sqlite')
mlflow.get_tracking_uri()

'sqlite:////home/jupyter/subreddit_clustering_i18n/mlflow_sync/djb-subclu-inference-tf-2-3-20210630/mlruns.db'

## Get list of experiments with new function

In [5]:
mlf.list_experiment_meta(output_format='pandas')

,experiment_id,name,artifact_location,lifecycle_stage
0,0,Default,./mlruns/0,active
1,1,fse_v1,gs://i18n-subreddit-clustering/mlflow/mlruns/1,active
2,2,fse_vectorize_v1,gs://i18n-subreddit-clustering/mlflow/mlruns/2,active
3,3,subreddit_description_v1,gs://i18n-subreddit-clustering/mlflow/mlruns/3,active
4,4,fse_vectorize_v1.1,gs://i18n-subreddit-clustering/mlflow/mlruns/4,active
5,5,use_multilingual_v0.1_test,gs://i18n-subreddit-clustering/mlflow/mlruns/5,active
6,6,use_multilingual_v1,gs://i18n-subreddit-clustering/mlflow/mlruns/6,active
7,7,use_multilingual_v1_aggregates_test,gs://i18n-subreddit-clustering/mlflow/mlruns/7,active
8,8,use_multilingual_v1_aggregates,gs://i18n-subreddit-clustering/mlflow/mlruns/8,active
9,9,v0.3.2_use_multi_inference_test,gs://i18n-subreddit-clustering/mlflow/mlruns/9,active


# Check whether we have access to a GPU

In [6]:
l_phys_gpus = tf.config.list_physical_devices('GPU')
# from tensorflow.python.client import device_lib

print(
    f"\nBuilt with CUDA? {tf.test.is_built_with_cuda()}"
    f"\nGPUs\n==="
    f"\nNum GPUs Available: {len(l_phys_gpus)}"
    f"\nGPU details:\n{l_phys_gpus}"
#     f"\n\nAll devices:\n===\n"
#     f"{device_lib.list_local_devices()}"
)


Built with CUDA? True
GPUs
===
Num GPUs Available: 1
GPU details:
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Load config with data to process

In [7]:
config_data_v040 = LoadHydraConfig(
    config_path="../config/data_text_and_metadata",
    config_name='v0.4.0_19k_top_subs_and_geo_relevant_2021_09_27',
#     config_name='top_subreddits_2021_07_16',
)
config_data_v040.config_dict

{'dataset_name': 'v0.4.0 inputs - Top Subreddits (no Geo) + Geo-relevant subs, comments: TBD',
 'bucket_name': 'i18n-subreddit-clustering',
 'folder_subreddits_text_and_meta': 'subreddits/top/2021-09-24',
 'folder_posts_text_and_meta': 'posts/top/2021-09-27',
 'folder_comments_text_and_meta': None}

In [8]:
mlflow_experiment_test = 'v0.4.0_use_multi_inference_test'
mlflow_experiment_full = 'v0.4.0_use_multi_inference'

bucket_name = config_data_v040.config_dict['bucket_name']
subreddits_path = config_data_v040.config_dict['folder_subreddits_text_and_meta']
posts_path = config_data_v040.config_dict['folder_posts_text_and_meta']
comments_path = None

# Call function to vectorize text

- Batch of: 3000 
- Limit characters to: 1000
Finally leaves enough room to use around 50% of RAM (of 60GB)

The problem is that each iteration takes around 3 minutes, which means whole job for GERMAN only will tka around 4:42 hours:mins...

## Test on a `sample` of posts & comments to make sure entire process works first (before running long job)

For subreddit only, we can expand to more than 1,500 characters.

HOWEVER - when scoring posts &/or comments, we're better off trimming to first ~1,000 characters to speed things up. We can increase the character len if results aren't great... this could be a hyperparameter to tune.

```
08:27:18 | INFO | "Start vectorize function"
08:27:18 | INFO | "  Local model saving directory: /home/jupyter/subreddit_clustering_i18n/data/models/use_multilingual/2021-07-29_0827"
08:27:18 | INFO | "Loading df_posts...
  gs://i18n-subreddit-clustering/posts/top/2021-07-16"
08:27:26 | INFO | "  0:00:07.773679 <- df_post time elapsed"
08:27:26 | INFO | "  (1649929, 6) <- df_posts.shape"
08:27:27 | INFO | "  Sampling posts down to: 2,500"
08:27:27 | INFO | "  (2500, 6) <- df_posts.shape AFTER sampling"
08:27:27 | INFO | "Load comments df..."
08:27:57 | INFO | "  (19200854, 6) <- df_comments shape"
08:28:08 | INFO | "Keep only comments that match posts IDs in df_posts..."
08:28:11 | INFO | "  (31630, 6) <- updated df_comments shape"
08:28:11 | INFO | "  Sampling COMMENTS down to: 5,100"
08:28:11 | INFO | "  (5100, 6) <- df_comments.shape AFTER sampling"
08:28:11 | INFO | "Load subreddits df..."
08:28:12 | INFO | "  (3767, 4) <- df_subs shape"
...
08:28:15 | INFO | "Getting embeddings in batches of size: 2000"
100%
2/2 [00:03<00:00, 1.67s/it]
08:28:19 | INFO | "  Saving to local... df_vect_subreddits_description..."
08:28:19 | INFO | "  Logging to mlflow..."
08:28:20 | INFO | "Vectorizing POSTS..."
08:28:20 | INFO | "Getting embeddings in batches of size: 2000"
100%
2/2 [00:00<00:00, 2.42it/s]
08:28:21 | INFO | "  Saving to local... df_vect_posts..."
08:28:21 | INFO | "  Logging to mlflow..."
08:28:22 | INFO | "Vectorizing COMMENTS..."
08:28:22 | INFO | "Getting embeddings in batches of size: 2000"
100%
3/3 [00:01<00:00, 1.95it/s]
08:28:24 | INFO | "  Saving to local... df_vect_comments..."
08:28:24 | INFO | "  Logging to mlflow..."
08:28:25 | INFO | "  0:01:06.542544 <- Total vectorize fxn time elapsed"

```

# Test new batching function

Most inputs will be the same.
However, some things will change:
- Add new parameter to sample only first N files (we'll process each file individually)

### Timing is super fast, even with a bigger sample size

```
11:40:06 | INFO | "Start vectorize function"
11:40:06 | INFO | "  Local model saving directory: /home/jupyter/subreddit_clustering_i18n/data/models/use_multilingual/2021-07-29_1140"
11:40:07 | INFO | "MLflow tracking URI: sqlite:////home/jupyter/subreddit_clustering_i18n/mlflow_sync/djb-subclu-inference-tf-2-3-20210630/mlruns.db"
11:40:07 | INFO | "  Saving config to local path..."
11:40:07 | INFO | "  Logging config to mlflow..."
11:40:08 | INFO | "Loading model use_multilingual..."
11:40:10 | INFO | "  0:00:02.417308 <- Load TF HUB model time elapsed"
11:40:10 | WARNING | "For TF-HUB models, the only preprocessing applied is lowercase()"
11:40:10 | INFO | "Load subreddits df..."
11:40:11 | INFO | "  0:00:00.519934 <- df_subs loading time elapsed"
11:40:11 | INFO | "  (3767, 4) <- df_subs shape"
11:40:11 | INFO | "Vectorizing subreddit descriptions..."
100%
2/2 [00:03<00:00, 1.65s/it]
11:40:15 | INFO | "  0:00:04.080246 <- df_subs vectorizing time elapsed"
...
11:40:16 | INFO | "Loading df_posts...
11:40:23 | INFO | "  0:00:06.460565 <- df_post loading time elapsed"
11:40:23 | INFO | "  (1649929, 6) <- df_posts.shape"
11:40:24 | INFO | "  Sampling posts down to: 9,500"
11:40:24 | INFO | "  (9500, 6) <- df_posts.shape AFTER sampling"
11:40:24 | INFO | "Vectorizing POSTS..."
100%
5/5 [00:03<00:00, 1.59it/s]
11:40:28 | INFO | "  0:00:03.774021 <- df_posts vectorizing time elapsed"
...
11:40:30 | INFO | "Load comments df..."
11:40:58 | INFO | "  (19200854, 6) <- df_comments shape"
11:41:10 | INFO | "Keep only comments that match posts IDs in df_posts..."
11:41:14 | INFO | "  (95313, 6) <- updated df_comments shape"
11:41:14 | INFO | "  Sampling COMMENTS down to: 19,100"
11:41:14 | INFO | "  (19100, 6) <- df_comments.shape AFTER sampling"
11:41:14 | INFO | "Vectorizing COMMENTS..."
100%
10/10 [00:05<00:00, 1.60it/s]
11:41:20 | INFO | "  0:00:06.239953 <- df_posts vectorizing time elapsed"
11:41:20 | INFO | "  Saving to local... df_vect_comments..."
11:41:20 | INFO | "    42.1 MB <- Memory usage"
11:41:20 | INFO | "       2	<- target Dask partitions	   30.0 <- target MB partition size"
11:41:21 | INFO | "  Logging to mlflow..."
11:41:23 | INFO | "  0:01:16.130234 <- Total vectorize fxn time elapsed"

```

# Run full with `lower_case=False`
Let's see if the current refactor is good enough or if I really need to manually batch files...

**answer**: no it wasn't good enough -- 60GB of RAM wasn't good enough for 19Million comments _lol_.

```
...
12:02:14 | INFO | "  (19168154, 6) <- updated df_comments shape"
12:02:14 | INFO | "Vectorizing COMMENTS..."
12:02:14 | INFO | "Getting embeddings in batches of size: 2100"
100%
9128/9128 [1:32:26<00:00, 1.97it/s]

<__array_function__ internals> in concatenate(*args, **kwargs)

MemoryError: Unable to allocate 36.6 GiB for an array with shape (512, 19168154) and data type float32
```


In [13]:
mlflow.end_run(status='KILLED')

vectorize_text_tf.vectorize_text_to_embeddings(
    model_name='use_multilingual',
    run_name=f"posts_as_comments_full_text-{datetime.utcnow().strftime('%Y-%m-%d_%H%M%S')}",
    mlflow_experiment=mlflow_experiment_test,
    
    tokenize_lowercase=False,
    
    bucket_name=bucket_name,
    subreddits_path=None,
    posts_path=None,  # posts_path
    comments_path=posts_path,

    # Hack: Rename cols so that I can process posts as a batch of comments
    col_post_id=None,
    col_comment_id='post_id',
    col_text_comment='text',
    col_text_comment_word_count='text_word_count',
    cols_index_comment=['subreddit_name', 'subreddit_id', 'post_id'],
    local_comms_subfolder_relative='df_vect_posts',
    mlflow_comments_folder='df_vect_posts_extra_text',
    cols_comment_text_to_concat=['flair_text', 'post_url_for_embeddings', 'text', 'ocr_inferred_text_agg_clean'],
    
    tf_batch_inference_rows=2400,
    tf_limit_first_n_chars=900,
    
    n_sample_comment_files=2,
    get_embeddings_verbose=True,
    
#     n_sample_posts=9500,
#     n_sample_comments=19100,
)

17:52:09 | INFO | "Start vectorize function"
17:52:09 | INFO | "  Local model saving directory: /home/jupyter/subreddit_clustering_i18n/data/models/use_multilingual/2021-09-28_175209"
17:52:09 | INFO | "MLflow tracking URI: sqlite:////home/jupyter/subreddit_clustering_i18n/mlflow_sync/djb-subclu-inference-tf-2-3-20210630/mlruns.db"
17:52:10 | INFO | "  Saving config to local path..."
17:52:10 | INFO | "  Logging config to mlflow..."
17:52:10 | INFO | "Loading model use_multilingual..."
17:52:13 | INFO | "  0:00:02.347350 <- Load TF HUB model time elapsed"
17:52:13 | WARNING | "For TF-HUB models, the only preprocessing applied is lowercase()"
17:52:13 | INFO | "** Procesing Comments files one at a time ***"
17:52:13 | INFO | "-- Loading & vectorizing COMMENTS in files: 2 --
Expected batch size: 2200"
17:52:13 | WARNING | "df_posts missing, so we can't filter comments without a post...
local variable 'df_posts' referenced before assignment"
  0%|          | 0/2 [00:00<?, ?it/s]17:52:13 |

In [9]:
mlflow.end_run(status='KILLED')

## Re-do with new batching logic
Trying to do all 19 million comments at once broke, sigh, so need to batch one file at a time.

### Re-run comments and log to non-test mlflow experiment


Besides file-batching, this job increased the row-batches from 2,000 to 6,100... unclear if this is having a negative impact. Maybe smaller batches are somehow more efficient?
Now that I'm reading one file at a time, it looks like speed is taking a big hit

Baseline when running it all in memory. It took `1:32:26`, but it ran out of memory (RAM).
The current ETA is around `2 hours`

```
# singe file, all in memory (results in OOM)
12:02:14 | INFO | "Vectorizing COMMENTS..."
12:02:14 | INFO | "Getting embeddings in batches of size: 2100"
100%
9128/9128 [1:32:26<00:00, 1.97it/s]


# one file at a time... slower, but we get results one file at a time...
16%
6/37 [21:11<1:49:46, 212.45s/it]
```


In [10]:
mlflow.end_run(status='KILLED')

vectorize_text_tf.vectorize_text_to_embeddings(
    model_name='use_multilingual',
    run_name=f"posts_as_comments_batch_concat_text-{datetime.utcnow().strftime('%Y-%m-%d_%H%M%S')}",
    mlflow_experiment=mlflow_experiment_full,
    
    tokenize_lowercase=False,
    
    bucket_name=bucket_name,
    subreddits_path=subreddits_path,
    posts_path=None,  # posts_path
    comments_path=posts_path,

    # Hack: Rename cols so that I can process posts as a batch of comments
    col_post_id=None,
    col_comment_id='post_id',
    col_text_comment='text',
    col_text_comment_word_count='text_word_count',
    cols_index_comment=['subreddit_name', 'subreddit_id', 'post_id'],
    local_comms_subfolder_relative='df_vect_posts',
    mlflow_comments_folder='df_vect_posts_extra_text',
    cols_comment_text_to_concat=['flair_text', 'post_url_for_embeddings', 'text', 'ocr_inferred_text_agg_clean'],
    
    tf_batch_inference_rows=2450,
    tf_limit_first_n_chars=900,
    
    n_sample_comment_files=None,
    
#     n_sample_posts=9500,
#     n_sample_comments=19100,
)

21:58:53 | INFO | "Start vectorize function"
21:58:53 | INFO | "  Local model saving directory: /home/jupyter/subreddit_clustering_i18n/data/models/use_multilingual/2021-09-28_215853"
21:58:53 | INFO | "MLflow tracking URI: sqlite:////home/jupyter/subreddit_clustering_i18n/mlflow_sync/djb-subclu-inference-tf-2-3-20210630/mlruns.db"
21:58:54 | INFO | "  Saving config to local path..."
21:58:54 | INFO | "  Logging config to mlflow..."
21:58:54 | INFO | "Loading model use_multilingual..."
21:59:03 | INFO | "  0:00:09.057354 <- Load TF HUB model time elapsed"
21:59:03 | WARNING | "For TF-HUB models, the only preprocessing applied is lowercase()"
21:59:03 | INFO | "Load subreddits df..."
21:59:05 | INFO | "  0:00:01.338779 <- df_subs loading time elapsed"
21:59:05 | INFO | "  (19262, 4) <- df_subs shape"
21:59:05 | INFO | "Vectorizing subreddit descriptions..."
21:59:05 | INFO | "Getting embeddings in batches of size: 2450"
  0%|                                                     | 0/8 [00

In [11]:
gc.collect()

55224

# Run full with `lower_case=True`

This one is expected to be a little slower because it'll call `.str.lower()` on each batch of text.

---

TODO: unsure if it's worth running this job in parallel while I do work on a separate VM... might be a big pain to manually sync the rows from metrics & params happening at the same time in two different VMs.



In [12]:
mlflow.end_run(status='KILLED')

vectorize_text_tf.vectorize_text_to_embeddings(
    model_name='use_multilingual',
    run_name=f"posts_as_comments_batch_concat_text_lowercase-{datetime.utcnow().strftime('%Y-%m-%d_%H%M%S')}",
    mlflow_experiment=mlflow_experiment_full,
    
    tokenize_lowercase=True,
    
    bucket_name=bucket_name,
    subreddits_path=subreddits_path,
    posts_path=None,  # posts_path
    comments_path=posts_path,

    # Hack: Rename cols so that I can process posts as a batch of comments
    col_post_id=None,
    col_comment_id='post_id',
    col_text_comment='text',
    col_text_comment_word_count='text_word_count',
    cols_index_comment=['subreddit_name', 'subreddit_id', 'post_id'],
    local_comms_subfolder_relative='df_vect_posts',
    mlflow_comments_folder='df_vect_posts_extra_text',
    cols_comment_text_to_concat=['flair_text', 'post_url_for_embeddings', 'text', 'ocr_inferred_text_agg_clean'],
    
    tf_batch_inference_rows=2450,
    tf_limit_first_n_chars=900,
    
    n_sample_comment_files=None,
    
#     n_sample_posts=9500,
#     n_sample_comments=19100,
)

23:22:44 | INFO | "Start vectorize function"
23:22:44 | INFO | "  Local model saving directory: /home/jupyter/subreddit_clustering_i18n/data/models/use_multilingual/2021-09-28_232244"
23:22:44 | INFO | "MLflow tracking URI: sqlite:////home/jupyter/subreddit_clustering_i18n/mlflow_sync/djb-subclu-inference-tf-2-3-20210630/mlruns.db"
23:22:44 | INFO | "  Saving config to local path..."
23:22:44 | INFO | "  Logging config to mlflow..."
23:22:45 | INFO | "Loading model use_multilingual..."
23:22:47 | INFO | "  0:00:02.176534 <- Load TF HUB model time elapsed"
23:22:47 | WARNING | "For TF-HUB models, the only preprocessing applied is lowercase()"
23:22:47 | INFO | "Load subreddits df..."
23:22:48 | INFO | "  0:00:00.693968 <- df_subs loading time elapsed"
23:22:48 | INFO | "  (19262, 4) <- df_subs shape"
23:22:48 | INFO | "Vectorizing subreddit descriptions..."
23:22:48 | INFO | "Getting embeddings in batches of size: 2450"
  0%|                                                     | 0/8 [00

In [ ]:
LEGACY

# Run full with lower_case=False (legacy fse/fasttext)

Time on CPU, only comments + subs:
```
13:29:07 | INFO | "  (1108757, 6) <- df_comments shape"
13:29:08 | INFO | "  (629, 4) <- df_subs shape"

13:45:11 | INFO | "  0:16:21.475036 <- Total vectorize fxn time elapsed"
```

In [18]:
mlflow.end_run(status='KILLED')

model, df_vect, df_vect_comments, df_vect_subs = vectorize_text_to_embeddings(
    model_name='use_multilingual',
    run_name='full_data-lowercase_false',
    mlflow_experiment=mlflow_experiment_full,
    
    tokenize_lowercase=False,
    subreddits_path='subreddits/de/2021-06-16',
    posts_path=None,  # 'posts/de/2021-06-16',
    comments_path='comments/de/2021-06-16',
    tf_batch_inference_rows=1500,
    tf_limit_first_n_chars=1100,
    n_sample_posts=None,
    n_sample_comments=None,
)

13:28:50 | INFO | "Start vectorize function"
13:28:50 | INFO | "  Local model saving directory: /home/jupyter/subreddit_clustering_i18n/data/models/use_multilingual/2021-07-01_1328"
13:28:50 | INFO | "Load comments df..."
13:29:07 | INFO | "  (1108757, 6) <- df_comments shape"
13:29:07 | INFO | "Keep only comments that match posts IDs in df_posts..."
13:29:07 | INFO | "df_posts missing, so we can't filter comments..."
13:29:07 | INFO | "Load subreddits df..."
13:29:08 | INFO | "  (629, 4) <- df_subs shape"
13:29:08 | INFO | "MLflow tracking URI: sqlite:////home/jupyter/mlflow/mlruns.db"
13:29:09 | INFO | "Loading model use_multilingual...
  with kwargs: None"


13:29:11 | INFO | "  0:00:02.282361 <- Load TF HUB model time elapsed"
13:29:11 | WARNING | "For TF-HUB models, the only preprocessing applied is lowercase()"
13:29:11 | INFO | "Vectorizing subreddit descriptions..."


13:29:13 | INFO | "  Saving to local... df_vect_subreddits_description..."
13:29:13 | INFO | "  Logging to mlflow..."


13:29:14 | INFO | "Vectorizing COMMENTS..."
13:29:14 | INFO | "Getting embeddings in batches of size: 1500"


  0%|          | 0/740 [00:00<?, ?it/s]

13:44:30 | INFO | "  Saving to local... df_vect_comments..."
13:44:49 | INFO | "  Logging to mlflow..."
13:45:11 | INFO | "  0:16:21.475036 <- Total vectorize fxn time elapsed"
